In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [6]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can be used to create high-quality content such as articles, social media posts, product descriptions, and more. This helps businesses save time and resources on content creation while maintaining consistency and quality.
2. **Marketing Automation**: Generative AI can generate personalized marketing materials, such as email templates, ad copy, and product descriptions, based on customer data and behavior.
3. **Customer Service Chatbots**: Generative AI-powered chatbots can provide 24/7 support to customers, answering common questions and routing complex issues to human representatives.
4. **Product Design**: Generative AI can help designers create new products, such as 3D models, prototypes, and even entire product lines, using machine learning algorithms to optimize design parameters.
5. **Virtual Assistants**: Generative AI-powered virtual assistants can be i

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [7]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and even entire books. This can help businesses reduce the time and cost associated with content creation.
2. **Product Design**: Generative AI can be used to design new products, such as jewelry, fashion items, or household goods. This can help businesses quickly prototype and test new designs without the need for extensive human involvement.
3. **Marketing and Advertising**: Generative AI can be used to generate personalized ads, product recommendations, and even entire marketing campaigns. This can help businesses tailor their marketing efforts to specific customer segments.
4. **Customer Service Chatbots**: Generative AI can be used to create chatbots that can understand and respond to customer inquiries in a more human-like way. This can he

## Alternative approach - using OpenAI python library to connect to Ollama

In [8]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can be used to automate content creation, such as generating news articles, product descriptions, and social media posts. This can help businesses save time and resources while maintaining consistency in their content.
2. **Digital Marketing**: Generative AI can be used to optimize online ads, generate ad copy, and create personalized email campaigns. It can also help analyze customer data and predict their behavior, enabling more effective marketing strategies.
3. **Design and Prototyping**: Generative AI can be used to generate designs for products, such as product labels, packaging, and branding materials. It can also create prototypes and simulations, reducing the need for physical prototyping and iterative design processes.
4. **Creative Writing and Storytelling**: Generative AI can be used to co-create stories, articles, and blog posts with h

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [9]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
# A class to represent a Webpage
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

# A function that writes a User Prompt that asks for summaries of websites:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

# See how this function creates exactly the format above
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# Call the Ollama local API.
def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

# A function to display this nicely in the Jupyter output, using markdown
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [10]:
display_summary("https://cnn.com")

This appears to be a sample issue of the CNN website, showcasing various news articles and features from around the world. I'll summarize some of the top headlines:

**World News**

* **Pope Francis**: The Pope has passed away at the age of 96, leaving behind a legacy of service and compassion.
* **Israel-Hamas War**: The conflict between Israel and Hamas has intensified, with both sides suffering losses and a human cost.
* **Ukraine-Russia War**: Russia has returned the body of a Ukrainian journalist who died in Russian detention, sparking concerns about Russian treatment of prisoners.

**US Politics**

* **Trump Administration**: Former President Donald Trump is rumored to be planning a comeback, with several high-profile officials announcing their resignation or departures.
* **TSAFIR ABAVOV**: Two Israeli officials have been accused of attempting to purchase the remains of two dead Palestinian men for thousands of dollars.

**Business and Technology**

* **Apple Tariffs**: The US government has imposed tariffs on Chinese tech giant Apple, with CEO Tim Cook stating that the tariffs could cost the company up to $900 million this quarter.
* **Meta's AI Assistant App**: Facebook parent Meta has launched an AI assistant app, further competing with OpenAI and Google in the emerging field of digital assistants.

**Health**

* **Whooping Cough Outbreak**: Cases of whooping cough are rising globally, with experts warning of the need for increased vaccination efforts.
* **Forever Chemicals Research**: Researchers have made gains in understanding how to build homes using fungi as a sustainable alternative material solution.

This is just a snapshot of some of the top news headlines from the CNN website. If you'd like to know more about any specific topic, feel free to ask!

In [11]:
display_summary("https://anthropic.com")

**Anthropic Website Summary**
==========================

Anthropic is a company that builds AI to serve humanity's long-term well-being. They aim to create tools with human benefit at their foundation, focusing on responsible AI development.

### News and Announcements

* **Claude 3.7 Sonnet**: Anthropic's most intelligent AI model is now available.
	+ Released in February 2025
* **Anthropic Economic Index**: New publication released on March 27, 2025, discussing societal impacts of large language models.
* **Alignment faking in large language models**: Blog post from December 18, 2024, exploring alignment science.
* **Introducing the Model Context Protocol**: Product update for November 25, 2024.

### AI Research and Products

Anthropic focuses on building powerful technologies with human benefit at their foundation. They provide various resources, including:

* Claude, an open-source AI platform
* Anthropic Academy, a learning platform
* Research overview, featuring the Anthropic Economic Index and more

The company's mission is to create tools that put safety at the frontier of AI development.

### Products and Pricing

Anthropic offers various products and pricing plans for customers, including:

* Claude Code
* Claude team plan
* Claude enterprise plan
* Claude education plan
* Claude apps
* Pricing plans for Claude.ai